<a href="https://colab.research.google.com/github/sameerdewan/atl-guardian/blob/main/mvp_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests opencv-python opencv-python-headless numpy pytz supervision inference flask pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.7/239.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import signal
import sys
import time
from datetime import datetime
import threading
import cv2
import numpy as np
import requests
import pytz
import supervision as sv
from inference import get_model
import matplotlib.pyplot as plt
from flask import Flask, Response, jsonify, send_file, render_template_string, request
from pyngrok import ngrok

# Configuration
IMAGE_URL = 'https://webcams.nyctmc.org/api/cameras/04e09ed5-2d97-4e29-8438-b87748850dbb/image'
FRAME_RATE = .5
VIDEO_SIZE = (640, 480)
OUTPUT_IMAGE = "annotated_image.jpg"
NGROK_AUTHTOKEN = "2hqxnQqMdvTNdeJpBd7G5OgRlIs_4x1G5iRf4bHwWLk9CZQFk"
NGROK_PORT = 2000
CITY_FEED_API = 'https://webcams.nyctmc.org/api/cameras'

# Initialize model
model = get_model(model_id="yolov8n-640")

# Initialize data collection
detection_data = {
    'timestamp': [],
    'person': [],
    'car': [],
    'bus': [],
    'truck': [],
    'motorcycle': [],
    'bike': []
}

def process_image(image_url):
    # Perform inference
    results = model.infer(
        image=image_url,
        confidence=0.1,
        iou_threshold=.5
    )

    detections = results[0].predictions

    # Filter detections for vehicles or people
    valid_classes = {'person', 'car', 'bus', 'truck', 'motorcycle', 'bike'}
    filtered_detections = [det for det in detections if det.class_name in valid_classes]

    # Fetch the image to annotate
    response = requests.get(image_url, stream=True)
    if response.status_code == 200:
        response.raw.decode_content = True
        image = np.asarray(bytearray(response.raw.read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    else:
        print(f"Failed to fetch image for annotation. Status code: {response.status_code}")
        return None

    # Annotate frame
    for det in filtered_detections:
        x1 = int(det.x - det.width / 2)
        y1 = int(det.y - det.height / 2)
        x2 = int(det.x + det.width / 2)
        y2 = int(det.y + det.height / 2)
        label = f"{det.class_name}: {det.confidence:.2f}"
        if det.class_name == 'person':
            color = (0, 0, 255)  # Red color for people
        else:
            color = (0, 255, 0)  # Green color for vehicles
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return image, filtered_detections

def setup_signal_handler():
    def signal_handler(sig, frame):
        sys.exit(0)

    signal.signal(signal.SIGINT, signal_handler)

def print_detection_data(detection_data):
    print("\nCurrent Detection Data:")
    header = "{:<30} {:<10} {:<10} {:<10} {:<10} {:<15} {:<10}".format(
        "Timestamp", "Person", "Car", "Bus", "Truck", "Motorcycle", "Bike"
    )
    print(header)
    print("-" * len(header))
    for i in range(len(detection_data['timestamp'])):
        row = "{:<30} {:<10} {:<10} {:<10} {:<10} {:<15} {:<10}".format(
            detection_data['timestamp'][i].astimezone(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d %H:%M:%S"),
            detection_data['person'][i],
            detection_data['car'][i],
            detection_data['bus'][i],
            detection_data['truck'][i],
            detection_data['motorcycle'][i],
            detection_data['bike'][i]
        )
        print(row)
    print("\n")

def main():
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
    public_url = ngrok.connect(NGROK_PORT).public_url
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{NGROK_PORT}\"")

    setup_signal_handler()

    try:
        while True:
            # Process the image URL directly
            annotated_image, detections = process_image(IMAGE_URL)

            if annotated_image is not None:
                # Resize the image to match the desired size
                annotated_image = cv2.resize(annotated_image, VIDEO_SIZE)

                # Save the annotated image
                cv2.imwrite(OUTPUT_IMAGE, annotated_image)

                # Collect data for the detected objects
                timestamp = datetime.now(pytz.utc)
                detection_data['timestamp'].append(timestamp)
                detection_data['person'].append(sum(1 for det in detections if det.class_name == 'person'))
                detection_data['car'].append(sum(1 for det in detections if det.class_name == 'car'))
                detection_data['bus'].append(sum(1 for det in detections if det.class_name == 'bus'))
                detection_data['truck'].append(sum(1 for det in detections if det.class_name == 'truck'))
                detection_data['motorcycle'].append(sum(1 for det in detections if det.class_name == 'motorcycle'))
                detection_data['bike'].append(sum(1 for det in detections if det.class_name == 'bike'))

                # Print the current detection data for debugging
                # print_detection_data(detection_data)

            # Wait for the next frame
            time.sleep(1 / FRAME_RATE)
    finally:
        print("Exiting...")

# Flask App
app = Flask(__name__)

@app.route('/detection_data', methods=['GET'])
def get_detection_data():
    return jsonify(detection_data)

@app.route('/annotated_image')
def annotated_image():
    return send_file(OUTPUT_IMAGE, mimetype='image/jpeg')

@app.route('/update_image_url', methods=['POST'])
def update_image_url():
    global IMAGE_URL
    IMAGE_URL = request.json.get('image_url')
    return jsonify({"message": "IMAGE_URL updated successfully", "new_url": IMAGE_URL})

@app.route('/')
def html():
    html_content = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <title>Detection Data Graph</title>
        <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
        <style>
            body {{
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 0;
                background-color: #f4f4f9;
                color: #333;
            }}
            .container {{
                max-width: 1200px;
                margin: 0 auto;
                padding: 20px;
            }}
            h2 {{
                text-align: center;
                color: #4CAF50;
            }}
            .chart-container {{
                display: flex;
                justify-content: center;
                align-items: center;
                margin-bottom: 30px;
                background: #fff;
                border-radius: 10px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
                padding: 20px;
            }}
            .image-container {{
                display: flex;
                justify-content: space-between;
                gap: 20px;
                margin-bottom: 30px;
            }}
            .image-container img {{
                width: 48%;
                border-radius: 10px;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            }}
            .dropdown-container {{
                display: flex;
                justify-content: center;
                margin-bottom: 20px;
            }}
            select {{
                padding: 10px;
                font-size: 16px;
                border-radius: 5px;
                border: 1px solid #ccc;
                box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <h2>Select Traffic Camera</h2>
            <div class="dropdown-container">
                <select id="cameraDropdown">
                    <option value="" disabled>Select a camera</option>
                </select>
            </div>
            <h2>Detection Data Graph</h2>
            <div class="chart-container">
                <canvas id="detectionChart"></canvas>
            </div>
            <h2>Live Images</h2>
            <div class="image-container">
                <img id="liveImage" src="{IMAGE_URL}" alt="Live Traffic Cam">
                <img id="annotatedImage" src="/annotated_image" alt="Annotated Traffic Cam">
            </div>
        </div>
        <script>
            const frameRate = {FRAME_RATE * 1000};
            let IMAGE_URL = '{IMAGE_URL}';
            let detectionData = {{ timestamp: [], person: [], car: [], bus: [], truck: [], motorcycle: [], bike: [] }};

            async function fetchCameras() {{
                const response = await fetch('/cameras');
                const cameras = await response.json();
                const dropdown = document.getElementById('cameraDropdown');
                cameras.forEach(camera => {{
                    const option = document.createElement('option');
                    option.value = camera.imageUrl;
                    option.textContent = camera.name;
                    dropdown.appendChild(option);
                }});
            }}

            async function fetchData() {{
                const response = await fetch('/detection_data');
                const data = await response.json();
                return data;
            }}

            function updateChart(chart, data) {{
                const labels = data.timestamp.map(ts => new Date(ts).toLocaleTimeString('en-US', {{ hour12: false }}));
                chart.data.labels = labels;
                chart.data.datasets[0].data = data.person;
                chart.data.datasets[1].data = data.car;
                chart.data.datasets[2].data = data.bus;
                chart.data.datasets[3].data = data.truck;
                chart.data.datasets[4].data = data.motorcycle;
                chart.data.datasets[5].data = data.bike;
                chart.update();
            }}

            async function setupChart() {{
                const ctx = document.getElementById('detectionChart').getContext('2d');
                const initialData = await fetchData();
                const chart = new Chart(ctx, {{
                    type: 'line',
                    data: {{
                        labels: initialData.timestamp.map(ts => new Date(ts).toLocaleTimeString('en-US', {{ hour12: false }})),
                        datasets: [
                            {{ label: 'Person', data: initialData.person, borderColor: 'red', fill: false }},
                            {{ label: 'Car', data: initialData.car, borderColor: 'blue', fill: false }},
                            {{ label: 'Bus', data: initialData.bus, borderColor: 'green', fill: false }},
                            {{ label: 'Truck', data: initialData.truck, borderColor: 'purple', fill: false }},
                            {{ label: 'Motorcycle', data: initialData.motorcycle, borderColor: 'orange', fill: false }},
                            {{ label: 'Bike', data: initialData.bike, borderColor: 'brown', fill: false }},
                        ]
                    }},
                    options: {{
                        responsive: true,
                        scales: {{
                            x: {{ title: {{ display: true, text: 'Time' }} }},
                            y: {{ title: {{ display: true, text: 'Count' }} }}
                        }}
                    }}
                }});
                setInterval(async () => {{
                    const newData = await fetchData();
                    updateChart(chart, newData);
                }}, frameRate);
            }}

            function updateImages() {{
              const liveImage = document.getElementById('liveImage');
              const annotatedImage = document.getElementById('annotatedImage');
              liveImage.src = IMAGE_URL + '?' + new Date().getTime();
              annotatedImage.src = '/annotated_image?' + new Date().getTime();
            }}

            document.getElementById('cameraDropdown').addEventListener('change', async function() {{
                IMAGE_URL = this.value;
                detectionData = {{ timestamp: [], person: [], car: [], bus: [], truck: [], motorcycle: [], bike: [] }};
                await fetch('/update_image_url', {{
                    method: 'POST',
                    headers: {{
                        'Content-Type': 'application/json'
                    }},
                    body: JSON.stringify({{ image_url: IMAGE_URL }})
                }});
                setupChart(); // Restart chart history
                updateImages(); // Update images immediately
            }});

            fetchCameras();
            setupChart();
            setInterval(updateImages, frameRate);
        </script>
    </body>
    </html>
    '''
    return render_template_string(html_content)

@app.route('/cameras', methods=['GET'])
def get_cameras():
    response = requests.get(CITY_FEED_API)
    if response.status_code == 200:
        return jsonify(response.json())
    else:
        return jsonify([]), response.status_code

def run_flask():
    app.run(port=NGROK_PORT, use_reloader=False)

# Start the Flask app in a separate thread
threading.Thread(target=run_flask).start()

if __name__ == "__main__":
    main()
